In [1]:
# dataset https://www.coursera.org/learn/ml-foundations/supplement/6DeQc/retrieving-wikipedia-articles-assignment

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("./input/people_wiki.csv")

In [3]:
df.head(2)

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...


# Top word count words for Elton John

In [4]:
elton=df[df["name"]=="Elton John"]

In [5]:
# count vecotirzer :D 
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

from sklearn.feature_extraction.text import CountVectorizer

In [6]:
count_vectorizer = CountVectorizer()
count = count_vectorizer.fit_transform(df['text'])

In [7]:
analyzer = count_vectorizer.build_analyzer()

In [19]:
# this is already a dict with words and number of occurrences
vocabulary = count_vectorizer.vocabulary_

In [15]:
def count_words(doc):
    dic = {}
    if analyzer(doc):
        M = count_vectorizer.fit_transform([doc]).toarray()[0]
        for word,index in vocabulary.items():
            dic[word] = M[index]
    return dic

In [16]:
elton['word_count'] = elton['text'].apply(count_words)

/home/vargas/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
elton

,URI,name,text,word_count
19923,<http://dbpedia.org/resource/Elton_John>,Elton John,sir elton hercules john cbe born reginald kenn...,"{'sir': 1, 'elton': 3, 'hercules': 1, 'john': ..."


In [23]:
# creating a table with dict of words and occurrences
elton_count_table = pd.DataFrame(elton['word_count'].values[0].items(),columns=['word','count'])
elton_count_table.sort_values(by='count',ascending=False).head()

,word,count
56,the,27
60,in,18
21,and,15
55,of,13
25,has,9


# Top TF-IDF words for Elton John

In [24]:
# https://en.wikipedia.org/wiki/Tf%E2%80%93idf
# term frequency–inverse document frequency, the tf–idf value increases proportionally to the number of 
# times a word appears in the document and is offset by the number of documents in the corpus that 
# contain the word, which helps to adjust for the fact that some words appear more frequently in general.

In [117]:
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer()
tf_idf_matrix = tf_idf_vectorizer.fit_transform(list(elton["text"]))
print( tf_idf_matrix.shape)

(1, 252)


In [118]:
# this is a matrix, and we need a list to append to the dataset
arrays_of_list = tf_idf_matrix.tolil().data
tf_idf= arrays_of_list.tolist()

In [120]:
#tf_idf[0]

In [121]:
# appending this list to the dataframe
elton_count_table["tf_idf"]=tf_idf[0]

In [122]:
elton_count_table.sort_values(by='tf_idf',ascending=False).head()

,word,count,tf_idf
227,openly,1,0.563848
135,number,2,0.375898
42,30,1,0.313249
176,charitable,1,0.271482
118,impact,1,0.187949


In [26]:
# ¿?

# The cosine distance between 'Elton John's and 'Victoria Beckham's articles (represented with TF-IDF) falls within which range?

In [124]:
elton
victoria = df[df['name'] == 'Victoria Beckham']

In [124]:
# in previous quizz is calculated "mathematically" the cosine distance from users

In [127]:
# we need to calculate again the tf-idf for victoria. I should have read this before and made a function,
# but lets be pragmatic
tf_idf_matrix = tf_idf_vectorizer.fit_transform(list(victoria["text"]))
arrays_of_list = tf_idf_matrix.tolil().data
tf_idf= arrays_of_list.tolist()
victoria_tf_idf=pd.DataFrame()
victoria_tf_idf["tf_idf"]=tf_idf[0]
victoria_tf_idf.sort_values(by='tf_idf',ascending=False).head()

In [152]:
victoria_tf_idf["tf_idf"].shape

(173,)

In [153]:
elton_count_table["tf_idf"].shape

(252,)

In [145]:
from sklearn.metrics.pairwise import cosine_distances

In [154]:
cosine_distances(elton_count_table["tf_idf"],victoria_tf_idf["tf_idf"])

ValueError: Expected 2D array, got 1D array instead:
array=[0.02088324 0.06264973 0.02088324 0.02088324 0.02088324 0.02088324
 0.02088324 0.02088324 0.02088324 0.02088324 0.02088324 0.04176649
 0.02088324 0.02088324 0.02088324 0.04176649 0.02088324 0.02088324
 0.02088324 0.02088324 0.02088324 0.04176649 0.02088324 0.02088324
 0.02088324 0.02088324 0.02088324 0.02088324 0.04176649 0.02088324
 0.02088324 0.02088324 0.02088324 0.06264973 0.02088324 0.02088324
 0.02088324 0.04176649 0.04176649 0.02088324 0.02088324 0.06264973
 0.31324867 0.02088324 0.02088324 0.02088324 0.04176649 0.04176649
 0.08353298 0.02088324 0.10441622 0.06264973 0.02088324 0.02088324
 0.02088324 0.06264973 0.02088324 0.02088324 0.04176649 0.08353298
 0.02088324 0.02088324 0.02088324 0.06264973 0.02088324 0.02088324
 0.02088324 0.02088324 0.02088324 0.02088324 0.02088324 0.02088324
 0.02088324 0.02088324 0.02088324 0.02088324 0.02088324 0.04176649
 0.02088324 0.02088324 0.02088324 0.02088324 0.02088324 0.02088324
 0.

(252,)

# The cosine distance between 'Elton John's and 'Paul McCartney's articles (represented with TF-IDF) falls within which range?

# Who is closer to 'Elton John', 'Victoria Beckham' or 'Paul McCartney'?

# Who is the nearest cosine-distance neighbor to 'Elton John' using raw word counts?

# Who is the nearest cosine-distance neighbor to 'Elton John' using TF-IDF?

# Who is the nearest cosine-distance neighbor to 'Victoria Beckham' using raw word counts?

# Who is the nearest cosine-distance neighbor to 'Victoria Beckham' using TF-IDF?